In [455]:
import numpy as np
import matplotlib.pyplot  as plt
import pandas as pd
import random

In [456]:
def sigmond(x):
    return 1/(1+np.exp(-x))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

In [524]:
###### 读取数据 
stock_data = pd.read_csv('stock_data_normal_u_30.csv',header=None)
stock_lable = pd.read_csv('stock_data_lable_normal_30.txt',header=None)

In [525]:
################# 抽取出成功的标签，使成功与失败的比例是1:1.2
x_data = stock_data.values
x_data = x_data.reshape(-1,66)
y_data = stock_lable.copy()
y_data['index'] = pd.DataFrame(np.arange(y_data.shape[0]))
y_data_1 = y_data[y_data[2]>0]
y_data_0 = y_data[y_data[2]==0]
rand_0_list = random.sample(np.arange(y_data_0.shape[0]).tolist(),int(y_data_1.shape[0]*1.2))
rand_0_np = np.array(rand_0_list)
y_data_0_random = y_data_0.loc[rand_0_np,:]

y_train = pd.concat(y_data_1,y_data_0_random)
index = y_train['index'].values
x_train = x_data.loc[index,:]
x_train = np.insert(x_train,0,1,axis=1)

19980
1642
1642
1369


In [498]:
lab_count = stock_lable.shape[0]
x = stock_data.loc[0:59999].values
print(x.shape)
x = x.reshape(-1,66)
print(x.shape)
y = stock_lable.loc[0:9999,2].values
y = y.reshape(-1,1)
print(y.shape)
#x = x / 2
x = np.insert(x,0,1,axis=1)

(60000, 11)
(10000, 66)
(10000, 1)


In [499]:
################# 抽取出成功的标签，使成功与失败的比例是1:1
def test_data_part(x_data,y_data):
    y_data['index'] = pd.DataFrame(np.arange(y_data.shape[0]))
    y_data_1 = y_data[y_data[2]>0]
    print(y_data_1.shape)
    print(y_data_1)
    index = y_data_1['index'].values
    print(index)
    x_data_1 = x_data[index]
    print(x_data_1)

In [434]:
def J_softmax(h,y):
    a = 1e-10
    b = y*np.log(h+a)
    return b.sum() / 60000

In [435]:
def J_sigmond(h,y):
    J = -y*np.log(h) - (1-y)*np.log(1-h)
    return J.sum() / 10000

In [436]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1.txt",delimiter=',')
w2 = np.loadtxt("init_w2.txt",delimiter=',')
w2 = w2.reshape(-1,1)
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 
n = 0.8
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10])

[[-0.65397757  0.22367608  0.30924261 -0.84598848  0.43765273]
 [-0.68750491 -0.21496053 -0.12832877  0.16931455 -0.50653401]
 [-0.3581152  -0.08942677 -0.09098002 -0.68911551  0.527162  ]
 [ 0.60997183  0.59494207 -0.31567412 -0.55302639  0.58082082]
 [-0.20312886 -0.83358934  0.18730978  0.7387345   0.88121403]
 [ 0.49482286 -0.15429002 -0.11693314  0.85685384 -0.61908754]
 [ 0.50001466 -0.66215733  0.52835968 -0.07026221 -0.31180939]
 [ 0.54848416  0.84468943 -0.73111227 -0.69457325  0.56632888]
 [-0.27914234  0.6610909   0.74562812  0.23827241  0.73464794]
 [-0.3531482  -0.79126145 -0.11991752 -0.05136866  0.39113913]]
[[ 0.21510757]
 [ 0.72417315]
 [ 0.56143739]
 [ 0.59901447]
 [-0.00920834]
 [ 0.42944708]
 [-0.43699503]
 [ 0.24706803]
 [-0.13887984]
 [ 0.3568151 ]]


In [288]:
"""
构建一个三层神经网络
一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段
取一般数据为训练样本数

数据归一化，分字段进行，然后 reshape(-1,1)
隐藏层 200 个神经元,输入到隐藏层，用reLu
输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy
w1 的 大小为 67*200
w2 的 大小为 201*1

"""

'\n构建一个三层神经网络\n一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段\n取一般数据为训练样本数\n\n数据归一化，分字段进行，然后 reshape(-1,1)\n隐藏层 200 个神经元,输入到隐藏层，用reLu\n输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy\nw1 的 大小为 67*200\nw2 的 大小为 201*1\n\n'

In [439]:
for i in range(1000):

    ############ 隐藏层输入 ############
    hid_input = np.matmul(x,w1)  ##### 10000* 200
    ############ 隐藏层输出 ############
    hid_output = np.tanh(hid_input)
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)   # 10000* 201
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ############ 最后一层的输出，即正向传播的输出值 ############
    #print(hpfs_input[0:10])
    hpfs_output = sigmond(hpfs_input)
    #print(hpfs_output[0:10])
    ############ 完成正向传播 ############

    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################

    d_cost_hyfs_input = hpfs_output - y
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / 10000
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    #print(d_cost_hyfs_input.shape,w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T) 
    d_cost_hid_input = d_cost_hid_out*(1-hid_output*hid_output)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    d_cost_w1 = np.matmul(x.T,d_cost_hid_input) / 10000
    print('----------cost is -------------',J_sigmond(hpfs_output,y))
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    #print('w1 最大值位置',np.where(w1==w1.max()),',w1 最大值',np.max(w1))
    #print('d_w1 最大值位置',np.where(d_cost_w1==d_cost_w1.max()),',d_w1 最大值',np.max(d_cost_w1))
    np.savetxt("init_w1.txt",w1,delimiter=',')
    np.savetxt("init_w2.txt",w2,delimiter=',')

print('----------------done--------------------------')

----------cost is ------------- 1.8125164862230347
----------cost is ------------- 2.1582976618825414
----------cost is ------------- 1.8113542429872855
----------cost is ------------- 2.156897730036083
----------cost is ------------- 1.8101917015285875
----------cost is ------------- 2.1554972125041227
----------cost is ------------- 1.8090288605253726
----------cost is ------------- 2.1540961080181344
----------cost is ------------- 1.807865718654823
----------cost is ------------- 2.152694415306771
----------cost is ------------- 1.806702274592813
----------cost is ------------- 2.1512921330957577
----------cost is ------------- 1.8055385270138522
----------cost is ------------- 2.1498892601078015
----------cost is ------------- 1.8043744745910286
----------cost is ------------- 2.1484857950624874
----------cost is ------------- 1.8032101159959486
----------cost is ------------- 2.147081736676184
----------cost is ------------- 1.8020454498986807
----------cost is ------------- 2.14

----------cost is ------------- 1.7172923001108693
----------cost is ------------- 2.0428903588588367
----------cost is ------------- 1.7161014222528552
----------cost is ------------- 2.0414384919893935
----------cost is ------------- 1.7149101305591552
----------cost is ------------- 2.03998592049623
----------cost is ------------- 1.7137184234422713
----------cost is ------------- 2.0385326426357624
----------cost is ------------- 1.7125262993097052
----------cost is ------------- 2.0370786566564947
----------cost is ------------- 1.7113337565639206
----------cost is ------------- 2.035623960798993
----------cost is ------------- 1.710140793602312
----------cost is ------------- 2.034168553295862
----------cost is ------------- 1.7089474088171814
----------cost is ------------- 2.0327124323717247
----------cost is ------------- 1.7077536005956957
----------cost is ------------- 2.0312555962432026
----------cost is ------------- 1.7065593673198634
----------cost is ------------- 2.02

----------cost is ------------- 1.6193458398831886
----------cost is ------------- 1.922843780449696
----------cost is ------------- 1.6181158218836222
----------cost is ------------- 1.9213284856982904
----------cost is ------------- 1.6168852427062401
----------cost is ------------- 1.9198123179394393
----------cost is ------------- 1.6156541002568119
----------cost is ------------- 1.9182952747235127
----------cost is ------------- 1.6144223924330574
----------cost is ------------- 1.9167773535904848
----------cost is ------------- 1.6131901171245646
----------cost is ------------- 1.9152585520698289
----------cost is ------------- 1.6119572722127173
----------cost is ------------- 1.913738867680421
----------cost is ------------- 1.6107238555706085
----------cost is ------------- 1.9122182979304356
----------cost is ------------- 1.6094898650629572
----------cost is ------------- 1.91069684031723
----------cost is ------------- 1.6082552985460243
----------cost is ------------- 1.9

----------cost is ------------- 1.517701353243498
----------cost is ------------- 1.797046335408742
----------cost is ------------- 1.516418225589955
----------cost is ------------- 1.7954512549960688
----------cost is ------------- 1.5151343315391737
----------cost is ------------- 1.7938550564748466
----------cost is ------------- 1.5138496679212545
----------cost is ------------- 1.7922577359466718
----------cost is ------------- 1.5125642315187653
----------cost is ------------- 1.790659289437484
----------cost is ------------- 1.5112780190572197
----------cost is ------------- 1.7890597128813557
----------cost is ------------- 1.5099910271931372
----------cost is ------------- 1.7874590021001975
----------cost is ------------- 1.5087032524991224
----------cost is ------------- 1.785857152778391
----------cost is ------------- 1.5074146914451951
----------cost is ------------- 1.7842541604310835
----------cost is ------------- 1.5061253403754813
----------cost is ------------- 1.78

----------cost is ------------- 0.7826040116506293
----------cost is ------------- 0.7828193994692151
----------cost is ------------- 0.7731654033629588
----------cost is ------------- 0.7734006726692544
----------cost is ------------- 0.7657270829243791
----------cost is ------------- 0.7660781569021162
----------cost is ------------- 0.7598192170638657
----------cost is ------------- 0.7603489790596171
----------cost is ------------- 0.7551186440233802
----------cost is ------------- 0.7558798389176933
----------cost is ------------- 0.7514117908065939
----------cost is ------------- 0.752460412195566
----------cost is ------------- 0.7485685796541978
----------cost is ------------- 0.7499732904176759
----------cost is ------------- 0.7465259536875645
----------cost is ------------- 0.7483768048844719
----------cost is ------------- 0.7452794872723247
----------cost is ------------- 0.7476982359171936
----------cost is ------------- 0.7448821769144794
----------cost is ------------- 

----------cost is ------------- 0.6990002411457363
----------cost is ------------- 0.7181849701847567
----------cost is ------------- 0.7253597892716857
----------cost is ------------- 0.7588113629057036
----------cost is ------------- 0.7696190805145998
----------cost is ------------- 0.8304754132759256
----------cost is ------------- 0.8477629235123857
----------cost is ------------- 0.9664544611737964
----------cost is ------------- 0.9992256735406343
----------cost is ------------- 1.2601536811070337
----------cost is ------------- 1.345314777747056
----------cost is ------------- 2.052258742295821
----------cost is ------------- 2.33780426004776
----------cost is ------------- 4.769725810009708
----------cost is ------------- 4.31928227176487
----------cost is ------------- 9.553894893283543
----------cost is ------------- 3.7035867815273544
----------cost is ------------- 6.420446888389944
----------cost is ------------- 4.169228742692425
----------cost is ------------- 7.9259561

----------cost is ------------- 2.591244506049422
----------cost is ------------- 3.778974061323534
----------cost is ------------- 2.388657901353716
----------cost is ------------- 3.359145932346455
----------cost is ------------- 2.178986926954158
----------cost is ------------- 2.943304125522196
----------cost is ------------- 1.958537233034104
----------cost is ------------- 2.52587598101641
----------cost is ------------- 1.7249257124768298
----------cost is ------------- 2.105252067679402
----------cost is ------------- 1.4799278157532811
----------cost is ------------- 1.6919895788249553
----------cost is ------------- 1.2363350202955832
----------cost is ------------- 1.3180367082866025
----------cost is ------------- 1.0217955415796494
----------cost is ------------- 1.0293742792623868
----------cost is ------------- 0.8644156560100511
----------cost is ------------- 0.8480391273406022
----------cost is ------------- 0.7679308258482185
----------cost is ------------- 0.7514254

In [449]:
### 先用训练集验证

index = random.randint(0,999)
x_train_test = x[index]
x_train_test = x_train_test.reshape(1,-1)
#x_train_test = np.insert(x_train_test,0,1,axis=0)
y_test = y[index]
print('--序号--',index)
print('----输入-----',x_train_test.shape)
hid_input = np.matmul(x_train_test,w1)
hid_output = np.tanh(hid_input)
hid_output = np.insert(hid_output,0,1,axis=1)
hpfs_input = np.matmul(hid_output,w2)
hpfs_output = sigmond(hpfs_input)
print('-----输出-----',y_test,hpfs_output)

--序号-- 393
----输入----- (1, 67)
-----输出----- [0] [[0.00858166]]


In [451]:
#index = random.randint(1501,2100)
error_data = pd.DataFrame(columns =['1','2','3'])
#index = 300
error_count = 0
for index in range(10000,19999):
    x_test = stock_data.loc[index*6:index*6+5].values
    x_test = x_test.reshape(1,-1)
    x_test = np.insert(x_test,0,1,axis=1)
    print('--序号--',index)
    #print('----输入-----',x_test)
    y_test = stock_lable.loc[index,2]
    hid_input = np.matmul(x_test,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=1)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    output = hpfs_output[0][0]
    y_value = y_test
    #print(output)
    if(output > 0.5 and y_value ==1) or (output<0.5 and y_value ==0):
        print('-----输出-----',y_test,hpfs_output)
    else:
        print('-----输出-----',y_test,hpfs_output)
        error_count = error_count + 1
        error_data.append(stock_lable.loc[index],ignore_index='True')
    #print('-----输出-----',y_test,hpfs_output)

--序号-- 10000
-----输出----- 0 [[0.00477366]]
--序号-- 10001
-----输出----- 1 [[0.98933631]]
--序号-- 10002
-----输出----- 0 [[0.01592353]]
--序号-- 10003
-----输出----- 0 [[0.00243467]]
--序号-- 10004
-----输出----- 0 [[2.88255187e-05]]
--序号-- 10005
-----输出----- 0 [[4.58360234e-06]]
--序号-- 10006
-----输出----- 0 [[0.00478195]]
--序号-- 10007
-----输出----- 1 [[0.99655284]]
--序号-- 10008
-----输出----- 0 [[0.00022381]]
--序号-- 10009
-----输出----- 0 [[4.7379199e-07]]
--序号-- 10010
-----输出----- 1 [[0.99274307]]
--序号-- 10011
-----输出----- 0 [[0.99999882]]
--序号-- 10012
-----输出----- 0 [[0.00110199]]
--序号-- 10013
-----输出----- 0 [[0.00436761]]
--序号-- 10014
-----输出----- 1 [[0.99831065]]
--序号-- 10015
-----输出----- 0 [[0.00043826]]
--序号-- 10016
-----输出----- 0 [[0.4935094]]
--序号-- 10017
-----输出----- 0 [[0.24496486]]
--序号-- 10018
-----输出----- 0 [[0.31343808]]
--序号-- 10019
-----输出----- 0 [[0.45003949]]
--序号-- 10020
-----输出----- 1 [[0.99998918]]
--序号-- 10021
-----输出----- 0 [[0.32617764]]
--序号-- 10022
-----输出----- 0 [[8.6838126e-06]

-----输出----- 0 [[0.36479299]]
--序号-- 10217
-----输出----- 1 [[0.9322461]]
--序号-- 10218
-----输出----- 0 [[7.73508333e-05]]
--序号-- 10219
-----输出----- 0 [[0.35759743]]
--序号-- 10220
-----输出----- 1 [[0.00128873]]
--序号-- 10221
-----输出----- 0 [[0.00094679]]
--序号-- 10222
-----输出----- 0 [[0.03346074]]
--序号-- 10223
-----输出----- 0 [[0.00180654]]
--序号-- 10224
-----输出----- 0 [[2.44789145e-06]]
--序号-- 10225
-----输出----- 0 [[0.07115361]]
--序号-- 10226
-----输出----- 0 [[4.82257419e-06]]
--序号-- 10227
-----输出----- 0 [[0.00547404]]
--序号-- 10228
-----输出----- 0 [[0.11080005]]
--序号-- 10229
-----输出----- 0 [[0.1296229]]
--序号-- 10230
-----输出----- 0 [[0.38053114]]
--序号-- 10231
-----输出----- 1 [[0.99993526]]
--序号-- 10232
-----输出----- 0 [[0.00287708]]
--序号-- 10233
-----输出----- 0 [[0.0109071]]
--序号-- 10234
-----输出----- 0 [[0.00115485]]
--序号-- 10235
-----输出----- 0 [[0.0036905]]
--序号-- 10236
-----输出----- 1 [[0.99741686]]
--序号-- 10237
-----输出----- 0 [[0.00071191]]
--序号-- 10238
-----输出----- 1 [[0.00094513]]
--序号-- 10239
---

-----输出----- 0 [[0.00019147]]
--序号-- 10437
-----输出----- 0 [[0.00208288]]
--序号-- 10438
-----输出----- 0 [[0.00231995]]
--序号-- 10439
-----输出----- 1 [[0.50812113]]
--序号-- 10440
-----输出----- 0 [[3.19450368e-05]]
--序号-- 10441
-----输出----- 0 [[1.41491784e-05]]
--序号-- 10442
-----输出----- 0 [[0.16969925]]
--序号-- 10443
-----输出----- 0 [[0.00067924]]
--序号-- 10444
-----输出----- 1 [[0.99655109]]
--序号-- 10445
-----输出----- 0 [[0.01304634]]
--序号-- 10446
-----输出----- 0 [[0.6660166]]
--序号-- 10447
-----输出----- 1 [[0.04414922]]
--序号-- 10448
-----输出----- 0 [[0.0018705]]
--序号-- 10449
-----输出----- 1 [[0.00136591]]
--序号-- 10450
-----输出----- 0 [[0.00329695]]
--序号-- 10451
-----输出----- 1 [[0.00938827]]
--序号-- 10452
-----输出----- 0 [[0.98367191]]
--序号-- 10453
-----输出----- 0 [[1.54172807e-05]]
--序号-- 10454
-----输出----- 1 [[0.99767586]]
--序号-- 10455
-----输出----- 0 [[0.00033725]]
--序号-- 10456
-----输出----- 0 [[0.00021625]]
--序号-- 10457
-----输出----- 0 [[0.34430804]]
--序号-- 10458
-----输出----- 0 [[0.03793037]]
--序号-- 10459
-

--序号-- 10759
-----输出----- 1 [[0.98094885]]
--序号-- 10760
-----输出----- 0 [[0.88211617]]
--序号-- 10761
-----输出----- 0 [[1.34176512e-06]]
--序号-- 10762
-----输出----- 0 [[0.82557709]]
--序号-- 10763
-----输出----- 1 [[0.0124054]]
--序号-- 10764
-----输出----- 0 [[0.00480801]]
--序号-- 10765
-----输出----- 0 [[0.00021834]]
--序号-- 10766
-----输出----- 1 [[0.95631556]]
--序号-- 10767
-----输出----- 0 [[0.00059912]]
--序号-- 10768
-----输出----- 0 [[0.00736748]]
--序号-- 10769
-----输出----- 0 [[0.0007305]]
--序号-- 10770
-----输出----- 0 [[0.01174353]]
--序号-- 10771
-----输出----- 0 [[0.00191304]]
--序号-- 10772
-----输出----- 1 [[0.99994007]]
--序号-- 10773
-----输出----- 0 [[0.03295984]]
--序号-- 10774
-----输出----- 0 [[0.01028315]]
--序号-- 10775
-----输出----- 0 [[3.45849433e-08]]
--序号-- 10776
-----输出----- 0 [[0.00613348]]
--序号-- 10777
-----输出----- 1 [[0.91395101]]
--序号-- 10778
-----输出----- 0 [[0.01731778]]
--序号-- 10779
-----输出----- 0 [[1.34529419e-08]]
--序号-- 10780
-----输出----- 0 [[0.02029806]]
--序号-- 10781
-----输出----- 0 [[9.36506108e-05

--序号-- 11008
-----输出----- 1 [[0.99833938]]
--序号-- 11009
-----输出----- 0 [[0.00216809]]
--序号-- 11010
-----输出----- 0 [[0.67311397]]
--序号-- 11011
-----输出----- 0 [[0.9377284]]
--序号-- 11012
-----输出----- 1 [[1.81748235e-05]]
--序号-- 11013
-----输出----- 0 [[0.05591107]]
--序号-- 11014
-----输出----- 0 [[0.00271487]]
--序号-- 11015
-----输出----- 1 [[0.00040501]]
--序号-- 11016
-----输出----- 1 [[0.00907098]]
--序号-- 11017
-----输出----- 0 [[0.21958278]]
--序号-- 11018
-----输出----- 0 [[0.88001942]]
--序号-- 11019
-----输出----- 0 [[0.67468105]]
--序号-- 11020
-----输出----- 0 [[0.44189945]]
--序号-- 11021
-----输出----- 0 [[0.00146962]]
--序号-- 11022
-----输出----- 1 [[0.95260138]]
--序号-- 11023
-----输出----- 0 [[0.04250788]]
--序号-- 11024
-----输出----- 0 [[2.68354942e-06]]
--序号-- 11025
-----输出----- 0 [[0.00509216]]
--序号-- 11026
-----输出----- 1 [[0.55365438]]
--序号-- 11027
-----输出----- 0 [[0.72748175]]
--序号-- 11028
-----输出----- 0 [[2.42205535e-07]]
--序号-- 11029
-----输出----- 1 [[0.99933897]]
--序号-- 11030
-----输出----- 0 [[0.00306586]]


...

-----输出----- 0 [[0.00050029]]
--序号-- 18772
-----输出----- 0 [[0.00168981]]
--序号-- 18773
-----输出----- 1 [[0.99753438]]
--序号-- 18774
-----输出----- 1 [[0.00589357]]
--序号-- 18775
-----输出----- 0 [[0.03311171]]
--序号-- 18776
-----输出----- 0 [[0.00025363]]
--序号-- 18777
-----输出----- 0 [[1.20921014e-05]]
--序号-- 18778
-----输出----- 0 [[0.01151199]]
--序号-- 18779
-----输出----- 0 [[0.95360589]]
--序号-- 18780
-----输出----- 1 [[0.89220538]]
--序号-- 18781
-----输出----- 1 [[0.06406756]]
--序号-- 18782
-----输出----- 0 [[4.23063869e-05]]
--序号-- 18783
-----输出----- 0 [[0.0014852]]
--序号-- 18784
-----输出----- 0 [[0.00052726]]
--序号-- 18785
-----输出----- 0 [[0.99884202]]
--序号-- 18786
-----输出----- 0 [[0.45214896]]
--序号-- 18787
-----输出----- 1 [[0.51635969]]
--序号-- 18788
-----输出----- 0 [[0.48355353]]
--序号-- 18789
-----输出----- 0 [[9.76508743e-06]]
--序号-- 18790
-----输出----- 0 [[0.00016388]]
--序号-- 18791
-----输出----- 0 [[0.00196678]]
--序号-- 18792
-----输出----- 0 [[0.59696802]]
--序号-- 18793
-----输出----- 1 [[0.99996615]]
--序号-- 18794


--序号-- 18966
-----输出----- 0 [[0.42311346]]
--序号-- 18967
-----输出----- 0 [[0.02275092]]
--序号-- 18968
-----输出----- 0 [[0.01148826]]
--序号-- 18969
-----输出----- 0 [[2.57562536e-05]]
--序号-- 18970
-----输出----- 1 [[0.7335593]]
--序号-- 18971
-----输出----- 0 [[6.5100369e-06]]
--序号-- 18972
-----输出----- 1 [[0.00015114]]
--序号-- 18973
-----输出----- 0 [[0.00011554]]
--序号-- 18974
-----输出----- 0 [[0.98475869]]
--序号-- 18975
-----输出----- 1 [[0.42978506]]
--序号-- 18976
-----输出----- 0 [[0.99998905]]
--序号-- 18977
-----输出----- 0 [[0.00143325]]
--序号-- 18978
-----输出----- 0 [[0.00174058]]
--序号-- 18979
-----输出----- 0 [[0.99131684]]
--序号-- 18980
-----输出----- 0 [[0.00915837]]
--序号-- 18981
-----输出----- 0 [[0.95255949]]
--序号-- 18982
-----输出----- 0 [[0.01666305]]
--序号-- 18983
-----输出----- 0 [[6.72928124e-05]]
--序号-- 18984
-----输出----- 0 [[0.01055716]]
--序号-- 18985
-----输出----- 0 [[5.6266099e-05]]
--序号-- 18986
-----输出----- 0 [[0.00479942]]
--序号-- 18987
-----输出----- 0 [[0.0001374]]
--序号-- 18988
-----输出----- 0 [[0.00021952]]

-----输出----- 0 [[0.00017]]
--序号-- 19331
-----输出----- 0 [[0.01785597]]
--序号-- 19332
-----输出----- 0 [[0.27886551]]
--序号-- 19333
-----输出----- 0 [[0.90628905]]
--序号-- 19334
-----输出----- 0 [[0.01837212]]
--序号-- 19335
-----输出----- 0 [[0.01569856]]
--序号-- 19336
-----输出----- 1 [[7.60448699e-06]]
--序号-- 19337
-----输出----- 0 [[0.32676585]]
--序号-- 19338
-----输出----- 0 [[0.18191712]]
--序号-- 19339
-----输出----- 0 [[0.00212656]]
--序号-- 19340
-----输出----- 0 [[0.48212649]]
--序号-- 19341
-----输出----- 0 [[0.9841988]]
--序号-- 19342
-----输出----- 1 [[0.99997413]]
--序号-- 19343
-----输出----- 0 [[0.00101965]]
--序号-- 19344
-----输出----- 1 [[0.00046167]]
--序号-- 19345
-----输出----- 1 [[0.01298896]]
--序号-- 19346
-----输出----- 1 [[0.00041266]]
--序号-- 19347
-----输出----- 0 [[0.99987747]]
--序号-- 19348
-----输出----- 0 [[0.9646041]]
--序号-- 19349
-----输出----- 0 [[0.93638085]]
--序号-- 19350
-----输出----- 1 [[0.90725553]]
--序号-- 19351
-----输出----- 0 [[0.0327418]]
--序号-- 19352
-----输出----- 0 [[0.00164538]]
--序号-- 19353
-----输出----- 

--序号-- 19600
-----输出----- 0 [[0.09197411]]
--序号-- 19601
-----输出----- 0 [[0.00182694]]
--序号-- 19602
-----输出----- 0 [[1.69768838e-05]]
--序号-- 19603
-----输出----- 1 [[0.88969587]]
--序号-- 19604
-----输出----- 0 [[6.02886028e-07]]
--序号-- 19605
-----输出----- 0 [[0.0030542]]
--序号-- 19606
-----输出----- 0 [[0.19155621]]
--序号-- 19607
-----输出----- 1 [[0.31830271]]
--序号-- 19608
-----输出----- 0 [[8.7778645e-05]]
--序号-- 19609
-----输出----- 0 [[5.30137529e-05]]
--序号-- 19610
-----输出----- 0 [[0.00059578]]
--序号-- 19611
-----输出----- 0 [[0.00041781]]
--序号-- 19612
-----输出----- 0 [[0.11315814]]
--序号-- 19613
-----输出----- 0 [[1.95953379e-06]]
--序号-- 19614
-----输出----- 0 [[0.06404515]]
--序号-- 19615
-----输出----- 0 [[0.01601104]]
--序号-- 19616
-----输出----- 0 [[0.0057799]]
--序号-- 19617
-----输出----- 0 [[0.36088089]]
--序号-- 19618
-----输出----- 0 [[0.301915]]
--序号-- 19619
-----输出----- 1 [[0.12901872]]
--序号-- 19620
-----输出----- 0 [[0.18699144]]
--序号-- 19621
-----输出----- 0 [[0.52181081]]
--序号-- 19622
-----输出----- 1 [[0.6412951

-----输出----- 0 [[0.00014889]]
--序号-- 19929
-----输出----- 0 [[0.01207258]]
--序号-- 19930
-----输出----- 0 [[0.00227296]]
--序号-- 19931
-----输出----- 1 [[0.03993418]]
--序号-- 19932
-----输出----- 0 [[0.07910137]]
--序号-- 19933
-----输出----- 0 [[0.00015912]]
--序号-- 19934
-----输出----- 0 [[0.0004705]]
--序号-- 19935
-----输出----- 0 [[8.82617826e-05]]
--序号-- 19936
-----输出----- 1 [[0.00035672]]
--序号-- 19937
-----输出----- 0 [[0.03407529]]
--序号-- 19938
-----输出----- 0 [[0.60621139]]
--序号-- 19939
-----输出----- 0 [[7.82450238e-05]]
--序号-- 19940
-----输出----- 0 [[0.20766965]]
--序号-- 19941
-----输出----- 1 [[0.21775118]]
--序号-- 19942
-----输出----- 0 [[7.18358487e-07]]
--序号-- 19943
-----输出----- 0 [[0.98940084]]
--序号-- 19944
-----输出----- 0 [[1.20320261e-05]]
--序号-- 19945
-----输出----- 1 [[0.9329785]]
--序号-- 19946
-----输出----- 0 [[0.00074548]]
--序号-- 19947
-----输出----- 0 [[0.00041329]]
--序号-- 19948
-----输出----- 0 [[5.94541449e-06]]
--序号-- 19949
-----输出----- 0 [[0.62080307]]
--序号-- 19950
-----输出----- 1 [[0.99998631]]
--序号--

In [452]:
print(error_data.shape)

(0, 3)


In [453]:
print(error_data.values)

[]


In [454]:
error_count

2223